In [ ]:
!nvidia-smi

Thu Aug 18 11:14:15 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    23W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Unzip the data scrapped**

In [ ]:
cd /content/drive/MyDrive/buy_me_that_look/data/Scrapped

/content/drive/MyDrive/buy_me_that_look/data/Scrapped


In [ ]:
!unzip '/content/drive/MyDrive/buy_me_that_look/data/Scrapped/men_trousers.zip'

# **Module 3 (Generate embeddings of images)**
In this module we will use some pre-trained and get embeddings from their last layer

In [ ]:
!pip install tensorflow==2.7.0 # version 2.9.1 tkharaj error f densenet
print(tf.__version__)

In [ ]:
print(tf.__version__)

2.7.0


In [ ]:
import tensorflow as tf
import datetime as dt
import cv2
import numpy as np

In [ ]:
tf.keras.backend.clear_session()
model_embedding = tf.keras.applications.DenseNet121(
    include_top=False, weights='imagenet', input_tensor=None, input_shape=(520,520,3),
    pooling=None,
)
def load_img(path):

  img = cv2.imread(path,cv2.IMREAD_UNCHANGED) 
  img = cv2.resize(img,(520,520),interpolation=cv2.INTER_AREA)
  img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  img = tf.image.convert_image_dtype(img,tf.float32)[tf.newaxis, ...] 
  return img 

def get_embeddings(path):
  img = load_img(path)
  op = model_embedding.predict(img,steps=1)
  op = tf.squeeze(op, axis=None, name=None)
  op = tf.reduce_mean(op, axis=(0,1), keepdims=False, name=None).numpy()
  #op = op.eval(session=tf.Session()) 
  op = op.tolist()
  return op

In [ ]:
model_embedding

In [ ]:
t = dt.datetime.now()
em1 = get_embeddings('/content/drive/MyDrive/buy_me_that_look/data/images_cropped/shorts/15bce24bfd_1.jpg')
print("Time taken to generate embeddings ",dt.datetime.now()-t)

Time taken to generate embeddings  0:00:10.660113


In [ ]:
print(em1)
print(len(em1))

[0.000374801573343575, 0.001481077168136835, 0.0052502453327178955, 0.002039755927398801, 0.22246836125850677, 0.06606936454772949, 0.0003952218685299158, 0.00541389686986804, 0.25444096326828003, 0.0001498980273026973, 0.00373202096670866, 0.01296369731426239, 0.000491945305839181, 0.010279430076479912, 0.07321421802043915, 5.511363269761205e-05, 0.000779995636548847, 0.21337932348251343, 0.0022312880028039217, 0.00029474147595465183, 0.0028453797567635775, 0.003906003898009658, 0.0004038996121380478, 0.0004311298835091293, 0.003308040089905262, 0.0004423598002176732, 0.0023211410734802485, 0.017343290150165558, 0.013674728572368622, 0.003836567047983408, 0.0024691345170140266, 0.0040062302723526955, 0.025156879797577858, 6.207451224327087e-05, 0.4617291986942291, 0.0011471612378954887, 0.00813283957540989, 0.04587765783071518, 0.0009219159837812185, 0.05173754692077637, 0.09709293395280838, 6.374288204824552e-05, 0.2512846291065216, 0.0002602159511297941, 0.0001410949043929577, 0.020

In [ ]:
"""
So I am taking pre-trained models and removing the top layer and just taking the embeddings. So before finalize 
DenseNet12 model I tried ResNet50,ResNet101, MobileNet, Iception but all models embeddings had an average sparsity of 20% to 30%
whereas DenseNet12 had only 4% average sparsity in embedding vector. 

So I choose DenseNet12 model for embedding.
"""

'\nSo I am taking pre-trained models and removing the top layer and just taking the embeddings. So before finalize \nDenseNet12 model I tried ResNet50,ResNet101, MobileNet, Iception but all models embeddings had an average sparsity of 20% to 30%\nwhereas DenseNet12 had only 4% average sparsity in embedding vector. \n\nSo I choose DenseNet12 model for embedding.\n'

# **Module 4 (Store embeddings)**
In this module we will store all our embeddings in Elastic Search so we can easily retrive them using similarity logic


In [ ]:
!pip install elasticsearch==7.10 #7.17 bedlet l erreur w  nahatli erreur mtaa star louwel "hosts" #7.10 hiya l sabha houwa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 321 kB 8.5 MB/s 


In [ ]:
from elasticsearch import Elasticsearch
import os
import pandas as pd
import json
from elasticsearch import helpers
import os
import numpy as np

In [ ]:
import pandas as pd
import json
from elasticsearch import helpers
import os
import numpy as np

In [ ]:
!pip3 install --upgrade requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.3 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [ ]:
import elasticsearch
elasticsearch.__version__

(7, 10, 0)

In [ ]:

os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/AAIC/CS2/"

In [ ]:
!kaggle datasets download -d shreyas90999/mycasestudy02ee

100% 11.1G/11.1G [05:09<00:00, 26.0MB/s]
100% 11.1G/11.1G [05:09<00:00, 38.5MB/s]


In [ ]:
!mkdir data
!unzip -q '/content/Mask_RCNN/mycasestudy02ee.zip' -d '/content/data'
!rm -rf '/content/mycasestudy02ee.zip'

unzip:  cannot find or open /content/Mask_RCNN/mycasestudy02ee.zip, /content/Mask_RCNN/mycasestudy02ee.zip.zip or /content/Mask_RCNN/mycasestudy02ee.zip.ZIP.


In [ ]:
#get data from json file
folder = []
json_file =[]

for (root,dirs,files) in os.walk('/content/drive/MyDrive/buy_me_that_look/data/Scrapped/', topdown=False):
  #print(dirs)
  for directory in dirs:
    for i in os.listdir(root+directory):
      #print(i)
      if i.endswith('.json'):
        folder.append(directory)
        json_file.append(i)
        #print(i)
print(folder)#categories name
print(json_file)#file names 

['men_t-shirts', 'men_t-shirts', 'men_t-shirts', 'men_t-shirts', 'men_t-shirts', 'men_t-shirts', 'men_t-shirts', 'men_t-shirts', 'men_t-shirts', 'men_t-shirts', 'men_t-shirts', 'men_t-shirts', 'men_t-shirts', 'men_t-shirts', 'men_t-shirts', 'men_t-shirts', 'men_t-shirts', 'men_t-shirts', 'men_t-shirts', 'men_t-shirts', 'men_t-shirts', 'men_t-shirts', 'men_t-shirts', 'men_t-shirts', 'men_t-shirts', 'men_t-shirts', 'men_t-shirts', 'men_t-shirts', 'men_t-shirts', 'men_t-shirts', 'men_t-shirts', 'men_t-shirts', 'men_t-shirts', 'men_t-shirts', 'men_t-shirts', 'men_t-shirts', 'men_t-shirts', 'men_t-shirts', 'men_t-shirts', 'men_t-shirts', 'men_t-shirts', 'men_t-shirts', 'men_t-shirts', 'men_t-shirts', 'men_t-shirts', 'men_t-shirts', 'men_t-shirts', 'men_t-shirts', 'men_t-shirts', 'men_t-shirts', 'men_trousers', 'men_trousers', 'men_trousers', 'men_trousers', 'men_trousers', 'men_trousers', 'men_trousers', 'men_trousers', 'men_trousers', 'men_trousers', 'men_trousers', 'men_trousers', 'men_tr

In [ ]:
'4c1a5fdc55.json' in json_file

False

In [ ]:
len(json_file)

170

In [ ]:
#create csv
super_cat = []
cat = []
info = []
file_name = []
url =[]
for i in range(len(json_file)):
  f = open('/content/drive/MyDrive/buy_me_that_look/data/Scrapped/' + folder[i] + '/' + json_file[i] )
  data = json.load(f)
  #for j in data['name']:
  j=data['name']
  #if folder[i]=='men_casual_shorts' or folder[i]=='women_casual_shoes' or folder[i]=='women_flats' or folder[i]=='women_heels':
  #  super_cat.append('foot_ware')
  #  cat.append(folder[i])
  #  info.append(json_file[i])
  #  file_name.append(j)
  #  url.append(data['url'])
  if folder[i]=='men_hoodie'or folder[i]=='men_t-shirts':
    super_cat.append('upper_ware')
    cat.append(folder[i])
    info.append(json_file[i])
    file_name.append(j)
    url.append(data['url'])
  if folder[i]=='men_casual_shorts' or folder[i]=='men_trousers':
    super_cat.append('lower_ware')
    cat.append(folder[i])
    info.append(json_file[i])
    file_name.append(j)
    url.append(data['url'])
  f.close()


print(len(super_cat))
#df = pd.DataFrame({ 'id':np.arange(len(file_name)),
#                    'super_cat':super_cat,
#                   'cat':cat,
#                   'info':info,
#                   'file_name':file_name,
#                   'url':url})

170


In [ ]:
df = pd.DataFrame({ 'id':np.arange(len(file_name)),
                    'super_cat':super_cat,
                   'cat':cat,
                   'info':info,
                   'file_name':file_name,
                   'url':url})

In [ ]:
df.shape

(170, 6)

In [ ]:
df.head()

,id,super_cat,cat,info,file_name,url
0,0,upper_ware,men_t-shirts,abdee3a327.json,abdee3a327.,https://encrypted-tbn0.gstatic.com/images?q=tb...
1,1,upper_ware,men_t-shirts,9bef3d2e8c.json,9bef3d2e8c.,https://encrypted-tbn0.gstatic.com/images?q=tb...
2,2,upper_ware,men_t-shirts,ac83465fdd.json,ac83465fdd.,https://encrypted-tbn0.gstatic.com/images?q=tb...
3,3,upper_ware,men_t-shirts,e1a598912c.json,e1a598912c.,https://encrypted-tbn0.gstatic.com/images?q=tb...
4,4,upper_ware,men_t-shirts,baffb59de9.json,baffb59de9.,https://encrypted-tbn0.gstatic.com/images?q=tb...


In [ ]:
cd /content/drive/MyDrive/buy_me_that_look/data

/content/drive/MyDrive/buy_me_that_look/data


In [ ]:
df.to_csv("/content/drive/MyDrive/buy_me_that_look/data/data.csv",index=False)

configure elastic search

In [ ]:
!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.10.0-linux-x86_64.tar.gz -q
!tar -xzf elasticsearch-7.10.0-linux-x86_64.tar.gz
!chown -R daemon:daemon elasticsearch-7.10.0
# start server
import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.10.0/bin/elasticsearch'], 
                  stdout=PIPE, stderr=STDOUT,
                  preexec_fn=lambda: os.setuid(1)  # as daemon
                 )
# wait a bit then test


In [ ]:
!pip install elasticsearch==7.10.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 321 kB 4.0 MB/s 


In [ ]:
!curl -X GET "localhost:9200/"

{
  "name" : "c9315c1ccea4",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "5knScVADSwOw0-6zLo_bEw",
  "version" : {
    "number" : "7.10.0",
    "build_flavor" : "default",
    "build_type" : "tar",
    "build_hash" : "51e9d6f22758d0374a0f3f5c6e8f3a7997850f96",
    "build_date" : "2020-11-09T21:30:33.964949Z",
    "build_snapshot" : false,
    "lucene_version" : "8.7.0",
    "minimum_wire_compatibility_version" : "6.8.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


In [ ]:
#!sudo systemctl start elasticsearch # ***** hethi tna7i erreur mtaa failed to connect

System has not been booted with systemd as init system (PID 1). Can't operate.


In [ ]:
import elasticsearch
from elasticsearch import Elasticsearch
from elasticsearch import helpers

es = Elasticsearch(timeout=2000000000)
es

#es = Elasticsearch(HOST="http://localhost", PORT=9200)
#es = Elasticsearch(hosts='http://localhost:9200',timeout=2000000000)

<Elasticsearch([{}])>

In [ ]:
create_query = {
    "mappings": {
        "properties": {
            "description_vector": {
                "type": "dense_vector",
                "dims": 1024
            }
        }
    }
}
#create index
es.indices.create(index="upper_ware", body=create_query)
#es.indices.create(index="foot_ware", body=create_query)
es.indices.create(index="lower_ware", body=create_query)
#resp = es.index(index="test-index", id=1, body=create_query)
#print(resp)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'lower_ware'}

******************************END

*******************

https://stackoverflow.com/questions/39288997/elasticsearch-root-mapping-definition-has-unsupported-parameters-index-not-a

Transfer data to Elastic Search instance

In [ ]:
#genrate docs and embedding to transfer to Elastic search
docs = []
c = 0
t = dt.datetime.now()
for row in df.iterrows():
  json_obj = {}
  path = root + row[1]['cat']+'/' + row[1]['file_name']+'jpg'
  print('path: ',path)
  em = get_embeddings(path)
  json_obj['_index']=row[1]['super_cat']
  json_obj['_id']=row[1]['id']
  json_obj['url'] = row[1]['url']
  json_obj['cat'] = row[1]['cat']
  json_obj['file_name'] = row[1]['file_name']+'jpg'
  json_obj['description_vector'] = em
  docs.append(json_obj)
  c+=1
  if c%50==0:
    print(c)
    print(dt.datetime.now()-t)



path:  /content/drive/MyDrive/buy_me_that_look/data/Scrapped/men_t-shirts/abdee3a327.jpg
path:  /content/drive/MyDrive/buy_me_that_look/data/Scrapped/men_t-shirts/9bef3d2e8c.jpg
path:  /content/drive/MyDrive/buy_me_that_look/data/Scrapped/men_t-shirts/ac83465fdd.jpg
path:  /content/drive/MyDrive/buy_me_that_look/data/Scrapped/men_t-shirts/e1a598912c.jpg
path:  /content/drive/MyDrive/buy_me_that_look/data/Scrapped/men_t-shirts/baffb59de9.jpg
path:  /content/drive/MyDrive/buy_me_that_look/data/Scrapped/men_t-shirts/06ea13e281.jpg
path:  /content/drive/MyDrive/buy_me_that_look/data/Scrapped/men_t-shirts/931974ff91.jpg
path:  /content/drive/MyDrive/buy_me_that_look/data/Scrapped/men_t-shirts/8ba9200afd.jpg
path:  /content/drive/MyDrive/buy_me_that_look/data/Scrapped/men_t-shirts/191fc00090.jpg
path:  /content/drive/MyDrive/buy_me_that_look/data/Scrapped/men_t-shirts/8411a03fbd.jpg
path:  /content/drive/MyDrive/buy_me_that_look/data/Scrapped/men_t-shirts/a996078e33.jpg
path:  /content/drive

In [ ]:
print(len(docs))#docs fiha embeddings mtaa tsawer scrapped kol 
print(docs[0:5])

170
[{'_index': 'upper_ware', '_id': 0, 'url': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTTAXlVr6nkkit-dhLg3tAxTo1_cSys8knjiw&usqp=CAU', 'cat': 'men_t-shirts', 'file_name': 'abdee3a327.jpg', 'description_vector': [0.0003226902917958796, 0.0018797204829752445, 0.004407485015690327, 0.0029584569856524467, 0.09023366123437881, 0.35837867856025696, 0.0003926013014279306, 0.0038386930245906115, 0.1512213796377182, 0.0002318576880497858, 0.002001778222620487, 0.013975270092487335, 0.0007873786962591112, 0.018467729911208153, 0.043124645948410034, 3.25160326610785e-05, 0.0010481795761734247, 0.2715810239315033, 0.002266589319333434, 0.0002482508425600827, 0.003669589292258024, 0.004113997332751751, 0.00045968213817104697, 0.0007305633625946939, 0.0022192238830029964, 0.0003926924546249211, 0.002071395516395569, 0.016210688278079033, 0.007005675230175257, 0.002311188727617264, 0.004328756593167782, 0.0034428685903549194, 0.009953448548913002, 0.00018391314370092005, 0.261955618858

https://elasticsearch-py.readthedocs.io/en/master/api.html

In [ ]:
#transfer all embeddings to Elastic Search
helpers.bulk(es, docs) 

(170, [])

In [ ]:
#Search query (we will now search for similar products)
def search_similar_image(query_vec,index_label):
  search_query = {
      "size": 20,
      "_source": {
          "includes": ["file_name","url"]
      },
      "query": {
          "script_score": {
              "query": {
                  "match_all": {}
              },
              "script": {
                  #"source": "cosineSimilarity(params.queryVector, 'description_vector') + 1.0",
                  "source": "1 / l2norm(params.queryVector,'description_vector') + 1.0",
                  "params": {
                      "queryVector": query_vec
                  }
              }
          }
      }
  }
  response = es.search(
      index= index_label,
      body=search_query
  )

  return response


In [ ]:
query_vec = get_embeddings('/content/drive/MyDrive/buy_me_that_look/data/images_cropped/short_sleeve_top/008f279374_0.jpg')

In [ ]:
query_vec

[0.00036044695298187435,
 0.0020557830575853586,
 0.0064949942752718925,
 0.0025272390339523554,
 0.04720815271139145,
 0.2769520580768585,
 0.00011140539572807029,
 0.003731706878170371,
 0.13053008913993835,
 0.0002914502692874521,
 0.001459899009205401,
 0.013029087334871292,
 0.000831086072139442,
 0.01884983293712139,
 0.033994436264038086,
 3.434687459957786e-05,
 0.001032956992276013,
 1.2211204767227173,
 0.0022420308087021112,
 0.00035020135692320764,
 0.0012389362091198564,
 0.004846411291509867,
 0.0004192637570668012,
 0.00014640777953900397,
 0.004138760268688202,
 0.0004872601421084255,
 0.002904106630012393,
 0.026607098057866096,
 0.007882600650191307,
 0.002694424707442522,
 0.006316702347248793,
 0.005644500255584717,
 0.02697807177901268,
 0.00034931671689264476,
 0.3040013611316681,
 0.0023190754000097513,
 0.005941097624599934,
 0.040707752108573914,
 0.0018299476942047477,
 0.07395119220018387,
 0.06459353119134903,
 0.0006411575013771653,
 0.481642484664917,
 0.0

In [ ]:
index_label = 'upper_ware'
response = search_similar_image(query_vec,index_label)

In [ ]:
#below are results of similar items that we added in Elastic search
response

{'took': 990,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 90, 'relation': 'eq'},
  'max_score': 1.1066078,
  'hits': [{'_index': 'upper_ware',
    '_type': '_doc',
    '_id': '34',
    '_score': 1.1066078,
    '_source': {'file_name': '40993c2c77.jpg',
     'url': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS1e2mRo7tSxrm924H0aFPfas1u8bib4Mz-fQ&usqp=CAU'}},
   {'_index': 'upper_ware',
    '_type': '_doc',
    '_id': '10',
    '_score': 1.1048263,
    '_source': {'file_name': 'a996078e33.jpg',
     'url': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR5XfS2u_NMyqGm4-T3MDRwvkC6Rs2KuB1gYA&usqp=CAU'}},
   {'_index': 'upper_ware',
    '_type': '_doc',
    '_id': '100',
    '_score': 1.1030462,
    '_source': {'file_name': '80397cb449.jpg',
     'url': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRs_9tS0PA8__kLo9HOzflxtwKOOXZPa7hLjA&usqp=CAU'}},
   {'_index': 'upper_ware',
    '_type': '_